In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from scipy.integrate import quad
from scipy.stats import norm

# ----------------------------------------------------------------------------
# 1. MODEL PARAMETERS FOR GAUSSIAN HJM
# ----------------------------------------------------------------------------
sigma0 = 0.01    # σ₀ in σ(t,T) = σ₀ · exp[−a·(T−t)]
a      = 0.1     # decay‐parameter a > 0

# ----------------------------------------------------------------------------
# 2. FORWARD‐RATE VOLATILITY σ(t,T)
#    σ(t,T) = σ₀ · exp[−a (T − t)]   for T ≥ t, else 0
# ----------------------------------------------------------------------------
def sigma(t, T):
    """
    Deterministic forward‐rate volatility σ(t,T).
      σ(t,T) = sigma0 * exp(−a * (T - t))   if T ≥ t,
              = 0.0                        if  T < t.
    Here t and T can be scalars or numpy arrays (broadcastable).
    """
    t_arr = np.asarray(t)
    T_arr = np.asarray(T)
    # For each pair, only nonzero if T >= t
    val = sigma0 * np.exp(-a * (T_arr - t_arr))
    return np.where(T_arr >= t_arr, val, 0.0)


# ----------------------------------------------------------------------------
# 3. BOND‐RETURN VOLATILITY ν(t,T) = − ∫_{u=t}^T σ(t,u) du
# ----------------------------------------------------------------------------
def nu(t, T):
    """
    Bond‐return volatility ν(t,T) = −∫_{u=t}^T σ(t,u) du.
    If T < t, returns 0.  Otherwise integrates numerically.
    """
    if np.isscalar(t) and np.isscalar(T):
        if T < t:
            return 0.0
        integrand = lambda u: sigma(t, u)
        val, _ = quad(integrand, t, T)
        return -val
    else:
        # Vectorized version when t, T are arrays
        t_arr = np.asarray(t)
        T_arr = np.asarray(T)
        result = np.zeros_like(np.broadcast(t_arr, T_arr), dtype=float)

        # For each element, integrate if T>=t
        for idx in np.ndindex(result.shape):
            ti = float(np.take(t_arr, idx))
            Ti = float(np.take(T_arr, idx))
            if Ti < ti:
                result[idx] = 0.0
            else:
                integrand = lambda u: sigma(ti, u)
                val, _ = quad(integrand, ti, Ti)
                result[idx] = -val
        return result


# ----------------------------------------------------------------------------
# 4. HJM DRIFT α(t,T) = σ(t,T) · ∫_{u=t}^T σ(t,u) du
# ----------------------------------------------------------------------------
def hjm_drift_alpha(t, T):
    """
    HJM drift α(t,T) = σ(t,T) * ∫_{u=t}^T σ(t,u) du.
    If T < t, returns 0 (since σ(t,T) = 0 in that region).
    """
    sigma_tT = sigma(t, T)
    # If T < t, sigma_tT is zero → drift = 0 anyway
    integrand = lambda u: sigma(t, u)
    if np.isscalar(t) and np.isscalar(T):
        if T < t:
            return 0.0
        val, _ = quad(integrand, t, T)
        return sigma_tT * val
    else:
        # Vectorized: integrate elementwise
        t_arr = np.asarray(t)
        T_arr = np.asarray(T)
        result = np.zeros_like(np.broadcast(t_arr, T_arr), dtype=float)
        for idx in np.ndindex(result.shape):
            ti = float(np.take(t_arr, idx))
            Ti = float(np.take(T_arr, idx))
            sigma_tTi = sigma(ti, Ti)
            if Ti < ti:
                result[idx] = 0.0
            else:
                val, _ = quad(integrand, ti, Ti)
                result[idx] = sigma_tTi * val
        return result


# ----------------------------------------------------------------------------
# 5. FORWARD‐RATE SDE (UNDER RISK‐NEUTRAL MEASURE Q):
#
#       df(t,T) = α(t,T) dt + σ(t,T) dW^*(t),
#       where α(t,T) = σ(t,T) ∫_{u=t}^T σ(t,u) du.
# ----------------------------------------------------------------------------

# We won’t simulate here; we just provide α(t,T) & σ(t,T) as callable functions.


# ----------------------------------------------------------------------------
# 6. BOND PRICE P(t,T) FROM A GIVEN FORWARD CURVE f(t,u):
#
#       P(t,T) = exp(−∫_{u=t}^T f(t,u) du),
#      so that dP/P = [r(t) − ∫_{u=t}^T α(t,u) du + ½ ‖v(t,T)‖²] dt + v(t,T) dW^*(t),
#      and by HJM drift‐condition  (no‐arbitrage)  ⇒  r(t) − ∫_{u=t}^T α(t,u) du + ½ ‖v(t,T)‖² = r(t).
#      Hence  dP/P = r(t) dt + v(t,T) dW^*(t).  (v = −∫_{u=t}^T σ(t,u) du)
# ----------------------------------------------------------------------------

def bond_price_t0_T(zero_curve_func, T):
    """
    P(0,T) = exp(− ∫_{u=0}^T r(0,u) du).  
    This is ONLY at t=0 (we use our interpolated zero curve).
    """
    if T <= 0:
        return 1.0
    integrand = lambda u: zero_curve_func(u)
    integ_val, _ = quad(integrand, 0.0, T)
    return np.exp(-integ_val)

# Example: show P(0,T) for a few T
for T0 in [0.5, 1.0, 2.0, 3.0, 5.0]:
    print(f"P(0,{T0:.1f}) = {bond_price_t0_T(r0_T, T0):.6f}")


# ----------------------------------------------------------------------------
# 7. RADON–NIKODYM DENSITY FOR T‐FORWARD MEASURE (Q^T):
#
#    dQ^T / dQ |_t  = exp(−∫_0^t r(s) ds) · P(t,T) / P(0,T).
#
#    We implement a function that returns the density at time‐t (scalar).
# ----------------------------------------------------------------------------
def radon_nikodym_T_forward(t, P_t_T, P0_T, discount_int_0_to_t):
    """
    Returns dQ^T/dQ |_t = exp(−∫_0^t r(s) ds) * [P(t,T)/P(0,T)].

    - t: scalar “current time”
    - P_t_T: P(t,T)  (scalar)
    - P0_T:   P(0,T)  (scalar)
    - discount_int_0_to_t: ∫_0^t r(s) ds  (scalar, from known r(0,s) or simulated path)
    """
    return np.exp(-discount_int_0_to_t) * (P_t_T / P0_T)


# ----------------------------------------------------------------------------
# 8. INTEGRATED VARIANCE I = ∫_0^{T0} [ν(s,T0) − ν(s,T1)]^2 ds
# ----------------------------------------------------------------------------
def integrated_variance_I(T0, T1):
    """
    Compute I(T0,T1) = ∫_{s=0}^{T0} [ν(s,T0) − ν(s,T1)]^2 ds.
    This is used in the closed‐form bond‐option formula.
    """
    integrand = lambda s: (nu(s, T0) - nu(s, T1))**2
    val, _ = quad(integrand, 0.0, T0)
    return val


# ----------------------------------------------------------------------------
# 9. GAUSSIAN HJM BOND‐OPTION FORMULAS (CALL & PUT)
#
#    Under Q^{T0}, the forward price F(0) = P(0,T1)/P(0,T0) is log‐normal with
#    variance I(T0,T1).  Hence the “Black‐type” formula:
#
#      d_{1,2} = [ ln( P(0,T1) / (K P(0,T0)) ) ± ½ I ] / √I,
#      p_call(0) = P(0,T1) Φ(d1) − K P(0,T0) Φ(d2),
#      p_put(0)  = K P(0,T0) Φ(−d2) − P(0,T1) Φ(−d1).
# ----------------------------------------------------------------------------
def gauss_hjm_call_price(zero_curve_func, T0, T1, K):
    """
    Price at t=0 of a European call on the T1‐bond, expiring at T0,
    strike K, under a one‐factor deterministic σ HJM.
    """
    # Discount factors at t=0
    P0_T0 = bond_price_t0_T(zero_curve_func, T0)
    P0_T1 = bond_price_t0_T(zero_curve_func, T1)

    # Integrated variance I
    I = integrated_variance_I(T0, T1)
    sqrtI = np.sqrt(I)

    # Log‐forward ratio
    H = P0_T1 / (K * P0_T0)
    lnH = np.log(H)

    # d1, d2
    d1 = (lnH + 0.5 * I) / sqrtI
    d2 = (lnH - 0.5 * I) / sqrtI

    # Call price
    price = P0_T1 * norm.cdf(d1) - K * P0_T0 * norm.cdf(d2)
    return price

def gauss_hjm_put_price(zero_curve_func, T0, T1, K):
    """
    Price at t=0 of a European put on the T1‐bond, expiring at T0,
    strike K, under a one‐factor deterministic σ HJM.
    """
    # Discount factors at t=0
    P0_T0 = bond_price_t0_T(zero_curve_func, T0)
    P0_T1 = bond_price_t0_T(zero_curve_func, T1)

    # Integrated variance I
    I = integrated_variance_I(T0, T1)
    sqrtI = np.sqrt(I)

    # Log‐forward ratio
    H = P0_T1 / (K * P0_T0)
    lnH = np.log(H)

    # d1, d2
    d1 = (lnH + 0.5 * I) / sqrtI
    d2 = (lnH - 0.5 * I) / sqrtI

    # Put price
    price = K * P0_T0 * norm.cdf(-d2) - P0_T1 * norm.cdf(-d1)
    return price


# ----------------------------------------------------------------------------
# 10. PUT‐CALL PARITY CHECK
#     p_call(0) − p_put(0)  ?=  P(0,T1) − K P(0,T0)
# ----------------------------------------------------------------------------
def check_put_call_parity(zero_curve_func, T0, T1, K):
    P0_T0 = bond_price_t0_T(zero_curve_func, T0)
    P0_T1 = bond_price_t0_T(zero_curve_func, T1)
    call_val = gauss_hjm_call_price(zero_curve_func, T0, T1, K)
    put_val  = gauss_hjm_put_price(zero_curve_func, T0, T1, K)
    lhs = call_val - put_val
    rhs = P0_T1 - K*P0_T0
    return lhs, rhs, call_val, put_val, P0_T0, P0_T1


# ----------------------------------------------------------------------------
# 11. EXAMPLE USAGE
# ----------------------------------------------------------------------------
if __name__ == "__main__":
    # Choose option parameters
    T0_ex = 1.0     # Option expiry (years)
    T1_ex = 3.0     # Underlying bond maturity (years)
    # ATM strike: so that forward price P(0,T1)/P(0,T0) = 1  →  K = P(0,T1)/P(0,T0)
    P0T0_ex = bond_price_t0_T(r0_T, T0_ex)
    P0T1_ex = bond_price_t0_T(r0_T, T1_ex)
    K_ex = P0T1_ex / P0T0_ex

    # Compute call & put prices
    call_price_ex = gauss_hjm_call_price(r0_T, T0_ex, T1_ex, K_ex)
    put_price_ex  = gauss_hjm_put_price(r0_T, T0_ex, T1_ex, K_ex)

    # Check parity
    lhs_parity, rhs_parity, _, _, _, _ = check_put_call_parity(r0_T, T0_ex, T1_ex, K_ex)

    print("=== Gaussian HJM Bond Option Example ===")
    print(f"Expiry T0 = {T0_ex:.2f}  ,  Bond‐maturity T1 = {T1_ex:.2f}")
    print(f"P(0,{T0_ex:.2f}) = {P0T0_ex:.6f},  P(0,{T1_ex:.2f}) = {P0T1_ex:.6f}")
    print(f"ATM Strike K = {K_ex:.6f}")
    print(f"Call Price (t=0): {call_price_ex:.6f}")
    print(f"Put  Price (t=0): {put_price_ex:.6f}")
    print(f"Put‐Call Parity LHS = {lhs_parity:.6f}  ≈  RHS = {rhs_parity:.6f}\n")

    # Display the HJM drift α(t,T) and volatility σ(t,T) for a few (t,T) pairs:
    print("Some α(t,T) and σ(t,T) values (HJM drift & vol):")
    sample_points = [(0.0, 1.0), (0.0, 3.0), (0.5, 1.5), (1.0, 2.0)]
    for (ti, Ti) in sample_points:
        si = sigma(ti, Ti)
        ai = hjm_drift_alpha(ti, Ti)
        nui = nu(ti, Ti)
        print(f"  t={ti:.1f}, T={Ti:.1f}  →  σ={si:.6f},  α={ai:.6f},  ν={nui:.6f}")

P(0,0.5) = 0.992528
P(0,1.0) = 0.984620
P(0,2.0) = 0.966572
P(0,3.0) = 0.946485
P(0,5.0) = 0.903030
=== Gaussian HJM Bond Option Example ===
Expiry T0 = 1.00  ,  Bond‐maturity T1 = 3.00
P(0,1.00) = 0.984620,  P(0,3.00) = 0.946485
ATM Strike K = 0.961270
Call Price (t=0): 0.006516
Put  Price (t=0): 0.006516
Put‐Call Parity LHS = 0.000000  ≈  RHS = 0.000000

Some α(t,T) and σ(t,T) values (HJM drift & vol):
  t=0.0, T=1.0  →  σ=0.009048,  α=0.000086,  ν=-0.009516
  t=0.0, T=3.0  →  σ=0.007408,  α=0.000192,  ν=-0.025918
  t=0.5, T=1.5  →  σ=0.009048,  α=0.000086,  ν=-0.009516
  t=1.0, T=2.0  →  σ=0.009048,  α=0.000086,  ν=-0.009516
